In [103]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from tqdm import tqdm
from ipywidgets import FloatProgress
# tqdm().pandas()
%matplotlib qt

In [151]:
fixpath = 'D:/honda data/Delhi - 2015 Nov/DL2CAT0433_Honda Amaze'
# folders = [ name for name in os.listdir('D:/honda data/percarpercitydata/') if os.path.isdir(os.path.join('D:/honda data/percarpercitydata/', name)) ]

# # paths = glob.glob('D:/honda data/percarpercitydata/Ahmedabad - 2016 Oct/')
# folders

In [152]:
amaze0433 = {}
paths = glob.glob(fixpath + '/*.csv')
key =0
for ind, file in tqdm(enumerate(paths)):
    raw_data = pd.read_csv(file, skiprows = [0], header = [0]) 
    if (raw_data['NE'] == 0).all():
        print("null")
        continue
    else:
        data = raw_data.copy(deep = True)
        data = data.reset_index(drop = True)
        start = []
        end = []
        df = pd.DataFrame()
        last_index = data.shape[0]
        i = 0
        val = True
        for x in range(last_index):
            if x == last_index-1:
                if val == False:
                     end.append(x)
            elif data['NE'][x] > 0:
                if val:
                    start.append(x)
                    val = False
            else:
                if val == False:
                    end.append(x-1)
                    val=True

        print(start, end)
        for se in range(len(start)):            
            df = data.iloc[start[se]:end[se]+1]
            time = []
            for t in range(len(df)):
                time.append(t*100)
            df['new_time'] = time
            df = df.reset_index(drop = True)
            amaze0433[key] = {}
            amaze0433[key]['data'] = df
            amaze0433[key]['path'] = file

            if len(start)>1:
                key +=1
        key += 1

0it [00:00, ?it/s]

null


1it [00:00,  1.85it/s]

[0] [402]


C:\Users\Poshita\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
2it [00:00,  2.06it/s]

...

43it [00:17,  2.16it/s]

null
[0] [563]


45it [00:17,  2.86it/s]

null


46it [00:17,  2.66it/s]


In [153]:
amaze0433.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])

In [154]:
k0433={}
for keys, item in tqdm(amaze0433.items()):
    data = item['data']  
    k0433[keys] = {}
    k0433[keys]['path'] = amaze0433[keys]['path']
    df = pd.DataFrame()

    speed = []
    time=[]
    rpm = []
    clutch=[]
    ac= []
    for ind, x in enumerate(data['new_time']):
        if x % 1000 == 0 :
            speed.append(data['VSP'].iloc[ind])
            rpm.append(data['NE'].iloc[ind])
            time.append(x)
            clutch.append(data['CLSW1D'].iloc[ind])
            ac.append(data['ACS'].iloc[ind])
        else:
            continue
    df['speed'] = speed
    df['time'] = time
    df['rpm'] = rpm
    df['clutch'] = clutch
    df['ac'] = ac
    df = df.dropna()
    df['speed'] = df['speed']
    df['time'] = df['time']/1000
    speed_diff = df['speed'] - df['speed'].shift(1)
    time_diff = df['time'] - df['time'].shift(1)
    acc = (speed_diff/time_diff) * 0.0283                          # converting km/h-s to g
    ha = 0
    hb = 0
    idle_time = 0
    idle = 0
    s = 0
    c = 0
    ac_idle = 0
    ac_drive = 0
    for x in acc:
        if x > 0.4:
            ha = ha+1
        elif x < -0.6:
            hb = hb+1
    k0433[keys]['Hard Acceleration'] = ha
    k0433[keys]['Hard Brake'] = hb
    val = True
    for x in range(len(df['speed'])-1):
        if df['speed'][x] == 0 and df['rpm'][x] > 650:               # Idling
            idle_time = idle_time+1
            if val:
                idle += 1
                val = False
            if df['clutch'][x] == 1:
                c += 1
            if df['ac'][x] == 1:
                ac_idle += 1
        elif df['speed'][x] != 0 and df['rpm'][x] > 650:
            if df['ac'][x] == 1:
                ac_drive += 1
            val = True

    k0433[keys]['Total Idle Time (in mins)'] = round(idle_time/60,0)
    k0433[keys]['Idle Instances'] = idle
    k0433[keys]['Clutch in Idling (in mins)'] = round(c/60,0)
    k0433[keys]['Ac in Idle(in mins)'] = round(ac_idle/60,0)
    k0433[keys]['Ac in Driving(in mins)'] = round(ac_drive/60,0)
#     k[keys]['idling_times'] = idle
    for x in df['speed']:
        if x>90:
            s = s+1
    k0433[keys]['Overspeed (in mins)'] = round(s/60,0)
    
    val = True
    cumsum=0
    i=0
    while(i<len(data['ENG_FUEL_USED'])-1):
        if i==(len(data['ENG_FUEL_USED'])-2):
            end = i+1
            val=True
        elif data['ENG_FUEL_USED'][i+1] - data['ENG_FUEL_USED'][i]>=0:
            if val:
                start = i
                val = False
        elif data['ENG_FUEL_USED'][i+1] - data['ENG_FUEL_USED'][i]<0:
            end = i
            val = True
        if val:
            cumsum += data['ENG_FUEL_USED'][end]- data['ENG_FUEL_USED'][start]
        i += 1
    
    k0433[keys]['Fuel Used (in litres)'] = round(cumsum/1000,2)
    k0433[keys]['Total Distance (in kms)'] = data['METER_ODO_DATA'].iloc[-1] - data['METER_ODO_DATA'].iloc[0]
    k0433[keys]['Total Time (in mins)'] = round(data['Smp.time'].iloc[-1]/(1000*60),0)
    k0433[keys]['Mileage (kmpl)'] = round(k0433[keys]['Total Distance (in kms)']/k0433[keys]['Fuel Used (in litres)'], 0)
    if k0433[keys]['Total Time (in mins)'] > 360:
        fd = 1
    else:
        fd = 0
    if idle_time == 0:
        ci = 0
    else:
        ci = c/idle_time
    x1 = 7/40; x2 = 7/40; x3 = 5/40; x4 = 5/40; x5 = 6/40; x6 = 5/40; x7 = 5/40
    score = (5 * (x1 * (ha/100) + x2 * (hb/100) + x3 * (idle_time/(60*k0433[keys]['Total Time (in mins)'])) + x4 * (idle/100) +
                  x5 * (ci) + x7 * (fd)))  # rf in between 5*(0)-5*(1))
    ds = 5 - (1.5 * (score))  # 1.5 weighing factor because 66.6%-100% is bad driving, since we have three categories

    if (score < 0):
        ds = 0
    elif (score > 5):
        ds = 5
    k0433[keys]['Driver Score'] = round(ds,1)

  3%|▎         | 1/36 [00:00<00:08,  4.09it/s]C:\Users\Poshita\Anaconda3\lib\site-packages\ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Poshita\Anaconda3\lib\site-packages\ipykernel_launcher.py:107: RuntimeWarning: divide by zero encountered in double_scalars
100%|██████████| 36/36 [00:29<00:00,  1.21it/s]


In [155]:
k0433

{0: {'path': 'D:/honda data/Delhi - 2015 Nov/DL2CAT0433_Honda Amaze\\A20150921111637a.csv',
  'Hard Acceleration': 0,
  'Hard Brake': 0,
  'Total Idle Time (in mins)': 1.0,
  'Idle Instances': 1,
  'Clutch in Idling (in mins)': 0.0,
  'Ac in Idle(in mins)': 0.0,
  'Ac in Driving(in mins)': 0.0,
  'Overspeed (in mins)': 0.0,
  'Fuel Used (in litres)': 0.01,
  'Total Distance (in kms)': 0.0,
  'Total Time (in mins)': 1.0,
  'Mileage (kmpl)': 0.0,
  'Driver Score': 4.4},
 1: {'path': 'D:/honda data/Delhi - 2015 Nov/DL2CAT0433_Honda Amaze\\A20150921114936a.csv',
  'Hard Acceleration': 0,
  'Hard Brake': 0,
  'Total Idle Time (in mins)': 0.0,
  'Idle Instances': 2,
  'Clutch in Idling (in mins)': 0.0,
  'Ac in Idle(in mins)': 0.0,
  'Ac in Driving(in mins)': 0.0,
  'Overspeed (in mins)': 0.0,
  'Fuel Used (in litres)': 0.0,
  'Total Distance (in kms)': 0.0,
  'Total Time (in mins)': 0.0,
  'Mileage (kmpl)': nan,
  'Driver Score': 5},
 2: {'path': 'D:/honda data/Delhi - 2015 Nov/DL2CAT0433_H

In [158]:
keys = []
for key, items in tqdm(k0433.items()):
    if items['Total Distance (in kms)'] < 2:
        keys.append(key)
        

100%|██████████| 36/36 [00:00<00:00, 3281.43it/s]


In [159]:
for key in keys:
    del k0433[key]

In [160]:
kdf0433 = pd.DataFrame(k0433)

In [161]:
kdf0433 = kdf0433.drop('path')
kdf0433 = kdf0433.transpose()

In [162]:
kdf0433 = kdf0433.apply(pd.to_numeric)
kdf0433

,Hard Acceleration,Hard Brake,Total Idle Time (in mins),Idle Instances,Clutch in Idling (in mins),Ac in Idle(in mins),Ac in Driving(in mins),Overspeed (in mins),Fuel Used (in litres),Total Distance (in kms),Total Time (in mins),Mileage (kmpl),Driver Score
5,0,0,2.0,19,1.0,2.0,7.0,0.0,0.20,2.0,9.0,10.0,3.9
11,0,0,3.0,27,3.0,2.0,16.0,0.0,0.56,9.0,28.0,16.0,3.6
12,0,0,7.0,29,5.0,6.0,17.0,0.0,0.53,6.0,30.0,11.0,3.7
25,0,0,0.0,4,0.0,0.0,0.0,0.0,0.24,5.0,11.0,21.0,3.8
26,0,0,13.0,39,2.0,2.0,13.0,68.0,7.77,169.0,151.0,22.0,4.3
28,0,0,2.0,10,1.0,1.0,66.0,43.0,4.47,88.0,73.0,20.0,4.2
29,0,0,6.0,34,3.0,0.0,24.0,22.0,3.59,78.0,89.0,22.0,4.1
30,0,0,3.0,9,1.0,0.0,6.0,0.0,0.34,5.0,16.0,15.0,4.2


In [163]:
kdf0433.corr()

,Hard Acceleration,Hard Brake,Total Idle Time (in mins),Idle Instances,Clutch in Idling (in mins),Ac in Idle(in mins),Ac in Driving(in mins),Overspeed (in mins),Fuel Used (in litres),Total Distance (in kms),Total Time (in mins),Mileage (kmpl),Driver Score
Hard Acceleration,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hard Brake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Idle Time (in mins),NaN,NaN,1.000000,0.842937,0.499057,0.374878,-0.044871,0.672822,0.728764,0.739642,0.815540,0.225835,0.387356
Idle Instances,NaN,NaN,0.842937,1.000000,0.708988,0.425201,-0.048750,0.422560,0.516516,0.529514,0.653494,0.086412,0.041719
Clutch in Idling (in mins),NaN,NaN,0.499057,0.708988,1.000000,0.758936,0.052024,-0.071850,0.007954,0.000000,0.155070,-0.293702,-0.376192
Ac in Idle(in mins),NaN,NaN,0.374878,0.425201,0.758936,1.000000,-0.007403,-0.102071,-0.103604,-0.122036,-0.046371,-0.599251,-0.460321
Ac in Driving(in mins),NaN,NaN,-0.044871,-0.048750,0.052024,-0.007403,1.000000,0.473344,0.431827,0.389319,0.341279,0.265540,0.326256
Overspeed (in mins),NaN,NaN,0.672822,0.422560,-0.071850,-0.102071,0.473344,1.000000,0.990012,0.984501,0.943876,0.635756,0.727158
Fuel Used (in litres),NaN,NaN,0.728764,0.516516,0.007954,-0.103604,0.431827,0.990012,1.000000,0.998646,0.979949,0.663697,0.713562
Total Distance (in kms),NaN,NaN,0.739642,0.529514,0.000000,-0.122036,0.389319,0.984501,0.998646,1.000000,0.983971,0.676147,0.713366


In [121]:
kdf0433[(kdf0433['Total Distance (in kms)'] > 5) & (kdf0433['Total Distance (in kms)'] < 20)][['Total Idle Time (in mins)',
        'Idle Instances', 'Clutch in Idling (in mins)', 'Ac in Idle(in mins)', 'Ac in Driving(in mins)', 'Overspeed (in mins)',
        'Fuel Used (in litres)', 'Total Distance (in kms)', 'Total Time (in mins)', 'Mileage (kmpl)', 'Driver Score']][:10].plot( 
        kind='bar', grid = True, title = 'City 6638 for Distance 5 - 20 (km)', figsize=(30, 15))
plt.xlabel("Trip ID")
plt.savefig('amaze0433_5_20_060420.png')

In [122]:
kdf0433[kdf0433['Total Distance (in kms)'] < 5][['Total Idle Time (in mins)', 'Idle Instances', 'Clutch in Idling (in mins)',
        'Ac in Idle(in mins)', 'Ac in Driving(in mins)', 'Overspeed (in mins)', 'Fuel Used (in litres)',
        'Total Distance (in kms)', 'Total Time (in mins)', 'Mileage (kmpl)', 'Driver Score']].plot(kind = 'bar', grid = True,
        figsize=(15, 10), title = 'City 6638 for Distance 2-5 (km)')
plt.xlabel("Trip ID")
plt.savefig('amaze0433_2_5_060420.png')

In [123]:
ax = kdf0433[kdf0433['Total Distance (in kms)'] >20][['Total Idle Time (in mins)', 'Idle Instances', 'Clutch in Idling (in mins)',
        'Ac in Idle(in mins)', 'Ac in Driving(in mins)', 'Overspeed (in mins)', 'Fuel Used (in litres)',
        'Total Distance (in kms)', 'Total Time (in mins)', 'Mileage (kmpl)', 'Driver Score']].plot(kind = 'bar', grid = True, figsize=(15, 10),
                                                                                                   title = 'City 6638 for Distance above 20 km')
for i in ax.patches:
    # get_x pulls left or right; get_height pushes up or down
    ax.text(i.get_x(), i.get_height()+1, \
            str(round((i.get_height()), 2)), fontsize=11, color='black',
                )
plt.xlabel("Trip ID")
plt.savefig('amaze0433_20_060420.png')

In [22]:
kdf0433 = kdf0433[kdf0433['Mileage (kmpl)']!= -np.inf]

In [23]:
kdf0433['Mileage (kmpl)'].unique()

array([18., 15., 14., 11., 12., 16., 20., 13., 17.,  8.,  9., 21., 22.,
       19.])

In [140]:
ax = kdf0433[['Driver Score', 'idle (%)', 'clutch (%)']].plot(kind = 'bar', legend = True, grid = True, figsize=(15, 10),
    title = 'City 9134 Driver Score with other factors')
ax.legend(['Driver Score', '% of clutch pressed during whole Idle Time', '% '])
for i in ax.patches:
    # get_x pulls left or right; get_height pushes up or down
    ax.text(i.get_x(), i.get_height()+1, \
            str(round((i.get_height()), 2)), fontsize=11, color='dimgrey',
                )
plt.xlabel("Trip ID")
# plt.savefig('amaze0433_clutch_idletime_ds_060420.png')


Text(0.5, 0, 'Trip ID')

In [132]:
kdf0433['clutch (%)'] = (kdf0433['Clutch in Idling (in mins)']/kdf0433['Total Idle Time (in mins)'])*100

In [137]:
kdf0433['idle (%)'] = (kdf0433['Total Idle Time (in mins)']/kdf0433['Total Time (in mins)'])*100

In [134]:
kdf0433['idle instances (%)'] = (kdf0433['Idle Instances']/kdf0433['Total Time (in mins)'])*100

In [127]:
ax = kdf0433[['Idle Instances', 'Total Distance (in kms)']].plot(kind = 'bar',
    grid = True, figsize=(15, 10), title = 'City 6638 Idle Instances with Total Distance Comparison')
for i in ax.patches:
    # get_x pulls left or right; get_height pushes up or down
    ax.text(i.get_x(), i.get_height()+1, \
            str(round((i.get_height()), 2)), fontsize=11, color='black',
                )
plt.xlabel("Trip ID")
plt.savefig('amaze0433_idleinstances_distance_060420.png')

In [129]:
ax = kdf0433[['Mileage (kmpl)', 'Total Distance (in kms)']].plot(kind = 'bar',
    grid = True, figsize=(15, 10), title = 'City 6638 Mileage and Total Distance Relationship')
for i in ax.patches:
    # get_x pulls left or right; get_height pushes up or down
    ax.text(i.get_x(), i.get_height()+1, \
            str(round((i.get_height()), 2)), fontsize=11, color='black',
                )
plt.xlabel("Trip ID")
plt.savefig('amaze0433_mileage_distance_060420.png')

In [145]:
ax = kdf0433[['clutch (%)', 'idle (%)', 'Fuel Used (in litres)', 'tot time']].plot( grid = True, figsize=(15, 10), title = 'City 6638 Mileage and Total Distance Relationship')
for i in ax.patches:
    # get_x pulls left or right; get_height pushes up or down
    ax.text(i.get_x(), i.get_height()+1, \
            str(round((i.get_height()), 2)), fontsize=11, color='black',
                )
plt.xlabel("Trip ID")
# plt.savefig('amaze0433_mileage_distance_060420.png')

Text(0.5, 0, 'Trip ID')

In [143]:
kdf0433['tot time'] = (kdf0433['Fuel Used (in litres)']/kdf0433['Total Time (in mins)'])*100

In [147]:
ax = kdf0433[['Mileage (kmpl)', 'Driver Score']].plot(kind = 'bar',
    grid = True, figsize=(15, 10), title = 'City 6638 Mileage with Driver Score')
for i in ax.patches:
    # get_x pulls left or right; get_height pushes up or down
    ax.text(i.get_x(), i.get_height()+1, \
            str(round((i.get_height()), 2)), fontsize=11, color='black',
                )
plt.xlabel("Trip ID")
plt.savefig('amaze0433_mileage_ds_060420.png')

In [150]:
ax = kdf0433[['clutch (%)', 'Clutch in Idling (in mins)', 'Total Idle Time (in mins)']].plot(kind = 'bar', legend = True,
    grid = True, figsize=(15, 10), title = 'City 6638 Clutch and Idling')
plt.legend(['% of time clutch is pressed in Idling', 'Clutch in Idling (mins)', 'Total Idle Time (mins)'])
for i in ax.patches:
    # get_x pulls left or right; get_height pushes up or down
    ax.text(i.get_x(), i.get_height()+1, \
            str(round((i.get_height()), 2)), fontsize=11, color='black',
                )
plt.xlabel("Trip ID")
plt.savefig('amaze0433_clutch_idling_060420.png')